In [1]:
from gensim.utils import any2unicode as unicode
from nltk.stem import SnowballStemmer
from sklearn.utils import resample
import os, re, sys, pandas, unidecode, string

import numpy
import emoji
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 

unable to import 'smart_open.gcs', disabling that module


In [4]:
# -----------------------------------------------------------------------------
# balance_data: 0 para retornar dataframe sin balancear o 1 para retornar
# dataframe balanceado.
# lematizacion: lematiza los textos usando nltk 0 para no lematizar y 1
# para retornar el texto lematizado.
# -----------------------------------------------------------------------------
def get_mourning_df(balance_data):
    from gensim.utils import any2unicode as unicode
    from nltk.stem import SnowballStemmer
    from sklearn.utils import resample
    import os, re, sys, pandas, unidecode

    mourning_folder = 'entrenamiento de modelos/datos mourning'
    mourning_df = pandas.DataFrame(columns=['text', 'lang', 'mourning'])
    path, subfolders, files_list = list(os.walk(mourning_folder))[0]
    files_list.sort()

    for i in range(len(files_list)):
        sys.stdout.write("\rPreparando df " + str(round(((i + 1) / (len(files_list))) * 100, 2)) + "%")
        sys.stdout.flush()
        file_name, file_ext = files_list[i].split(".")

        if file_ext == 'csv':
            file_path = path + "/" + file_name + "." + file_ext
            df = pandas.read_csv(file_path, encoding='utf8', dtype=str, engine='python')
            numero_de_archivo = int(file_name.split("_")[0])

            if numero_de_archivo == 1 or numero_de_archivo == 2:
                df = df.filter(['text', 'lang', 'mourning'])
                df['mourning'] = df.mourning.map({'4': '0', '1': '1'})
                mourning_df = mourning_df.append(df)

            if numero_de_archivo == 3:
                df = df.filter(['text', 'lang', 'tag'])
                df.columns = ['text', 'lang', 'mourning']
                df['mourning'] = df.mourning.map({'no mourning': '0', 'mourning': '1'})
                mourning_df = mourning_df.append(df)

            if numero_de_archivo == 4:
                df = df.filter(['tweet', 'lang', 'mourning'])
                df.columns = ['text', 'lang', 'mourning']
                df['mourning'] = df.mourning.map({'no mourning': '0', 'mourning': '1'})
                mourning_df = mourning_df.append(df)

    del df
    print("")
    mourning_df.dropna()
    mourning_df.drop_duplicates(subset=['text'], inplace=True)
    mourning_df = mourning_df.loc[mourning_df['mourning'].isin(['1', '0'])]
    mourning_df = mourning_df.loc[mourning_df['lang'].isin(['es', 'en'])]
    mourning_df.reset_index(drop=True, inplace=True)

    for i, row in mourning_df.iterrows():
        sys.stdout.write("\rNormalizando df " + str(round(((i + 1) / (mourning_df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        mourning_df.at[i, 'text'] = (
            re.sub('\s+', ' ',
                   re.sub(' +', ' ',
                          re.sub("http\S+", "",
                                 re.sub(r'\b(?=\w*[j])[aeiouj]{2,}\b', 'jajaja',
                                        re.sub(r'[\b@]\w+\s{1}', '', str(mourning_df.at[i, 'text'])
                                               )))))).strip()

    print("")

    if balance_data == 1:
        mourning_df["Sello"] = 0
        for i, row in mourning_df.iterrows():
            sys.stdout.write(
                "\rCreando sellos de balanceamiento " +
                str(round(((i + 1) / (mourning_df.shape[0])) * 100, 2))
                + "%"
            )
            sys.stdout.flush()
            mourning_df.at[i, 'sello'] = str(mourning_df.at[i, 'lang']) + '_' + str(mourning_df.at[i, 'mourning'])
        print("\nBalanceando df")
        min_len1 = int(min(mourning_df['sello'].value_counts()))
        df_0 = resample(mourning_df[mourning_df.sello == 'es_0'], replace=False, n_samples=min_len1, random_state=1)
        df_1 = resample(mourning_df[mourning_df.sello == 'es_1'], replace=False, n_samples=min_len1, random_state=1)
        df_2 = resample(mourning_df[mourning_df.sello == 'en_0'], replace=False, n_samples=min_len1, random_state=1)
        df_3 = resample(mourning_df[mourning_df.sello == 'en_1'], replace=False, n_samples=min_len1, random_state=1)
        mourning_df = pandas.concat([df_0, df_1, df_2, df_3])
        mourning_df = mourning_df.filter(['text', 'lang', 'mourning'])
        mourning_df.reset_index(drop=True, inplace=True)
        
    print("Df mourning entregado\n")
    mourning_df.reset_index(drop=True, inplace=True)
    return mourning_df


In [5]:
df = get_mourning_df(1)

Preparando df 100.0%
Normalizando df 12.34%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Normalizando df 31.97%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Normalizando df 52.17%

...

Creando sellos de balanceamiento 60.84%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Creando sellos de balanceamiento 81.66%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Creando sellos de balanceamiento 100.0%
Balanceando df
Df mourning entregado



In [6]:
#--------------------
#Se separa la data por idiomas
#--------------------

df_en = df[df['lang']=="en"]
df_es = df[df['lang']=="es"]

In [7]:
#----------------- Lexicones
#Lectura de SentiWordNet Obtenido de 
#https://www.nltk.org/_modules/nltk/corpus/reader/sentiwordnet.html

# SentiWordNet[word] = {POS,	ID,	PosScore,	NegScore}
contador = 0
SentiWordNet = dict()
for lines in open('entrenamiento de modelos/data lexicon/SentiWordNet_3.0.0.txt'):
    if lines.startswith('#'):
        continue
    line = lines.split('\t')
    palabra = line[4].split('#')[0]
    if (palabra in SentiWordNet) or (palabra==''):
        continue
    else:
        SentiWordNet[palabra]={'POS': line[0], 'ID': line[1], 'PosScore': line[2], 'NegScore': line[3]}

In [8]:
# AFFIN[word] = sentiment
AFFIN = dict()
for lines in open('entrenamiento de modelos/data lexicon/AFFIN-111.txt'):
    AFFIN[lines.split('\t')[0]]=(lines.split('\t')[1]).split('\n')[0]

In [9]:

"""
se divide es el texto del tweet mediante el uso de 
TweetTokenizer() y se normalizan las risas, se eliminan las menciones
y se lematizan las palabras
""" 


def tokenizar(text, idioma):
    tweet_tokenizer = TweetTokenizer()
    palabras = []
    lema = {'es': SnowballStemmer('spanish'), 'en': SnowballStemmer('english')}
    
    text = re.sub(r'[\b@]\w+\s{1}', '', text)#Quitar menciones
    words = tweet_tokenizer.tokenize(text)
    for w in words:
        if re.match(r'\W', w)==None:
            
            p = unidecode.unidecode(w).lower() #Quitar Acentos minuscula
            p = re.sub(r'\b(?=\w*[j])[aeiouj]{2,}\b', 'jajaja', p) #Normalizar Risa
            p = re.sub(r'\b(?=\w*[h])[aeiouh]{2,}\b', 'jajaja', p) #Normalizar Risa
            p = lema[idioma].stem(p)
            palabras.append(p)
        elif re.match(r'#', w)!=None:
            palabras.append(unidecode.unidecode(w).lower())
        elif  w in emoji.UNICODE_EMOJI:
            palabras.append(w)
    return palabras


In [10]:
"""
se crea el lexicon basado en los tweets etiquetados para morning y no mourning
se usa la la fórmula P = P(word|mourning)*P(word)
"""

def lexicon_mornig(df):
    lexicon = dict()
    morning = [[],[]] # [no morning, morning]
    
    cont=0
    
    cont_morning = dict()
    cont_no_morning = dict()
    cont_total = dict()
    
    vocabulario = set([])
    
    i=0
    for index, row in df.iterrows():
        i+=1
        sys.stdout.write("\rCreando lexicon df " + str(round(((i) / (df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        
        cont += 1
                
        palabras = tokenizar(row['text'], row['lang'])  
        
        vocabulario|=set(palabras)
        morning[int(row['mourning'])].append(set(palabras))    
        #print(row['text'])
        #print(palabras)
        
    for v in morning[1]:
        for m in v:
            cont_morning[m]=1 if m not in cont_morning else cont_morning[m]+1
            cont_total[m]=1 if m not in cont_total else cont_total[m]+1
            
    for v in morning[0]:
        for m in v:
            cont_no_morning[m]=1 if m not in cont_morning else cont_morning[m]+1
            cont_total[m]=1 if m not in cont_total else cont_total[m]+1
            
    for w in vocabulario:
        if cont_total[w] > 5: 
            lexicon[w]=0
            if w in cont_no_morning:
                lexicon[w] -= (cont_no_morning[w]/len(morning[0]))*(cont_total[w]/cont) 
                
            if w in cont_morning:
                lexicon[w] += (cont_morning[w]/len(morning[1]))*(cont_total[w]/cont) 
    return lexicon

In [11]:
"""
Para vectorizar se reemplaza cada palabra con el valor generado del lexicón creado
y se agregan los valores obtenido de diferentes lexicones
"""

def vectorizar(df, lexicon):    
    print()
    keys = tuple(lexicon.keys())
    y = dict()
    for a in keys:
        y[a]=list(numpy.zeros(df.shape[0]))
    y['SentiWordNet_neg'] = 0
    y['SentiWordNet_pos'] = 0
    y['AFFIN'] = 0
    i=0
    
    for index, row in df.iterrows():
        sys.stdout.write("\rVectorizando df " + str(round(((i+1) / (df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        v = dict()
        palabras = tokenizar(row['text'], row['lang']) 
        
        sum_swn_neg = 0
        sum_swn_pos = 0
        
        affin = 0
        
        for w in palabras:
            if w in lexicon:
                y[w][i]=lexicon[w]
                
            if w in SentiWordNet.keys():
                y['SentiWordNet_neg'] += float(SentiWordNet[w]['NegScore'])
                y['SentiWordNet_pos'] += float(SentiWordNet[w]['PosScore'])
                
            if w in AFFIN.keys():
                y['AFFIN'] += float(AFFIN[w])
        i+=1
    
    vector = pandas.DataFrame.from_dict(y)    
    return vector


In [12]:
"""
Se crea el lexicón y se vectoriza los tweets en español
"""

lex_es = lexicon_mornig(df_es)
vector_es = vectorizar(df_es, lex_es).values

Creando lexicon df 35.93%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Creando lexicon df 95.56%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Vectorizando df 100.0%

In [13]:
from sklearn.cluster import KMeans
"""
Se entrena el modelo K-Means y se evalúa
"""

real_es = list(df_es['mourning'])
model = KMeans(n_clusters=2)
model.fit(vector_es)
predic_es = model.fit_predict(vector_es)

In [21]:
"""Generación de tabla pivote"""

df_comparison_es = pandas.DataFrame({'predic_es': predic_es, 'real_es': real_es})
tabla_pivot_es = pandas.crosstab(df_comparison_es['real_es'], df_comparison_es['predic_es'])
tabla_pivot_es

predic_es,0,1
real_es,,
0,2097,2078
1,2490,1685


In [20]:
from sklearn.metrics import classification_report
"""Generación de métricas del modelo"""
print("\nResultados ")
print(classification_report([int(i) for i in real_es], predic_es))


Resultados 
              precision    recall  f1-score   support

           0       0.46      0.50      0.48      4175
           1       0.45      0.40      0.42      4175

    accuracy                           0.45      8350
   macro avg       0.45      0.45      0.45      8350
weighted avg       0.45      0.45      0.45      8350



In [16]:
"""
Se crea el lexicón y se vectoriza los tweets en ingles
"""

lex_en = lexicon_mornig(df_en)
vector_en = vectorizar(df_en, lex_en).values

Creando lexicon df 100.0%
Vectorizando df 100.0%

In [17]:
from sklearn.cluster import KMeans
"""
Se entrena el modelo K-Means y se evalúa
"""

real_en = list(df_en['mourning'])
model_en = KMeans(n_clusters=2)
model_en.fit(vector_en)
predic_en = model_en.fit_predict(vector_en)

In [18]:
"""Generación de tabla pivote"""
df_comparison_en = pandas.DataFrame({'predic_en': predic_en, 'real_en': real_en})
tabla_pivot = pandas.crosstab(df_comparison_en['predic_en'], df_comparison_en['real_en'])
tabla_pivot

real_en,0,1
predic_en,,
0,2302,1959
1,1873,2216


In [19]:
from sklearn.metrics import classification_report
"""Generación de métricas del modelo"""
print("\nResultados ")
print(classification_report([int(i) for i in real_en], predic_en))


Resultados 
              precision    recall  f1-score   support

           0       0.54      0.55      0.55      4175
           1       0.54      0.53      0.54      4175

    accuracy                           0.54      8350
   macro avg       0.54      0.54      0.54      8350
weighted avg       0.54      0.54      0.54      8350

